# Convolutional Neural Network Vignette Primary Document
Kaitlyn, Oscar, Nini, Johanna
12/6/2025



## Introduction 

Convolutional Neural Networks (CNNs) are a special type of neural networks designed for processing grid-structured data, particularly images. Unlike traditional fully-connected neural networks, CNNs use the spatial structure of images through localized connectivity and parameter sharing. This vignette explores CNN process and performance through implementation of a multi-class butterfly species classifier.

## Limitations of Fully Connected NNs

Traditional neural networks face significant challenges when processing image data. For a 128×128 pixel RGB image (which is what our butterfly image dataset consists of), the input dimensionality is 49,152 (128 × 128 × 3). A single fully-connected hidden layer with 256 neurons requires 12,583,936 parameters. This parameter explosion leads to computational inefficiency and increases the risk of overfitting. Additionally, fully-connected networks lack translation equivariance. Identical patterns appearing in different spatial locations have to be learned independently, requiring substantially more training data to achieve comparable performance to CNNs.

## CNN Architecture Principles

CNNs address these limitations through two fundamental design principles:

### Parameter Sharing:

Small learnable filters, called kernels, are applied across the entire input via convolution operations. The same filter weights are used at every spatial location, drastically reducing parameter count while enabling position-invariant feature detection.

### Hierarchical Feature Learning:

Stacked convolutional layers create hierarchical representations. Initial layers extract low-level features (edges, textures), intermediate layers identify mid-level patterns (shapes, object parts), and deep layers recognize high-level semantic concepts.

## CNN Layer Components

### Convolutional Layers

Convolutional layers apply learnable filters through discrete convolution operations. Each filter performs element-wise multiplication with the receptive field and sums the results, producing activation maps that indicate feature presence at different spatial locations.

### Activation Functions

Non-linear activation functions, typically ReLU (Rectified Linear Unit: f(x) = max(0,x)), are applied element-wise following convolution operations. Non-linearity is essential for learning complex mappings beyond linear transformations.

### Pooling Layers

Pooling operations perform spatial downsampling. Max pooling selects the maximum value within each pooling window, reducing spatial dimensions while retaining salient features. This provides computational efficiency, partial translation invariance, and regularization.

### Flatten Layers

Flatten operations reshape multi-dimensional feature maps into one-dimensional vectors, enabling transition from convolutional feature extraction to fully-connected classification layers.

### Fully-Connected Layers

Dense layers with full connectivity between consecutive layers aggregate extracted features for final classification. These layers function identically to traditional neural network layers.

### Dropout

Dropout applies stochastic regularization by randomly deactivating neurons during training with specified probability. This prevents co-adaptation of features and reduces overfitting.

### Output Layer

The final layer uses softmax activation to produce normalized probability distributions over classes.

## Dataset Description

The dataset comprises butterfly images spanning 75 species classes. The training set contains 6,499 images, split into 5,199 training samples and 1,300 validation samples (80/20 ratio). The test set contains 2,786 images. All images are resized to 128×128 pixels and processed in batches of 32. Stratified sampling ensures proportional class representation across training and validation splits. This is critical for maintaining class balance in multi-class problems with potentially uneven class distributions.

## Model Architecture

The implemented CNN follows a sequential architecture:

### Input Layer:

128×128×3 (RGB images)

### Convolutional Block 1:

32 filters (3×3), ReLU activation, followed by 2×2 max pooling

### Convolutional Block 2:

64 filters (3×3), ReLU activation, followed by 2×2 max pooling

### Convolutional Block 3:

128 filters (3×3), ReLU activation, followed by 2×2 max pooling

### Flatten Layer:

Converts 3D feature maps to 1D vector

### Dense Layer:

256 units, ReLU activation Dropout Layer: 50% dropout rate

### Output Layer:

75 units (one per class), softmax activation

The model contains 6,535,307 trainable parameters.

## Depth Analysis

An experiment examining network depth trained models with 1, 2, 3, and 4 convolutional blocks:

### 1 Block:

Validation accuracy of 31.6%. Insufficient representational capacity for complex multi-class classification.

### 2 Blocks:

Validation accuracy of 55.7%. Additional depth enables more discriminative feature learning.

### 3 Blocks:

Validation accuracy of 67.2%. Three-layer hierarchy provides adequate feature abstraction for this task.

These results demonstrate that network depth directly impacts feature abstraction capability. However, excessive depth may lead to overfitting or training difficulties without sufficient data or regularization.

## Training Results

The model was trained for 10 epochs using the Adam optimizer and categorical cross-entropy loss. Training progression showed:

Epoch 1: 17.8% validation accuracy

Epoch 10: 67.2% validation accuracy

Training accuracy reached approximately 88-89% for the 2-block model, indicating some degree of overfitting. The training-validation accuracy gap suggests potential benefit from additional regularization or larger training datasets.

## Prediction Pipeline

Test set predictions follow this procedure:

1. Forward pass through trained model

2. Softmax output produces probability distribution over 75 classes

3. Class with maximum probability selected via argmax operation

4. Numeric class indices mapped to species labels

5. Results exported to CSV with filename and predicted species

# Conclusion

CNNs provide substantial advantages over fully-connected architectures for image classification tasks through parameter sharing and spatial structure exploitation. The butterfly classifier achieved 67.2% accuracy across 75 classes, demonstrating effective feature learning from training data. Network depth proved critical for performance, with deeper architectures enabling more sophisticated feature hierarchies. Further improvements could be obtained through transfer learning, additional regularization techniques, or larger training datasets.

# Code Summary

## Library Imports and Data Loading

In [3]:
!pip install tensorflow


                                              0.0/331.8 MB ? eta -:--:--
                                              0.1/331.8 MB 3.2 MB/s eta 0:01:44
                                              0.3/331.8 MB 2.9 MB/s eta 0:01:53
                                              0.4/331.8 MB 3.1 MB/s eta 0:01:47
                                              0.7/331.8 MB 3.9 MB/s eta 0:01:25
                                              1.0/331.8 MB 5.0 MB/s eta 0:01:07
                                              1.5/331.8 MB 5.7 MB/s eta 0:00:58
                                              2.3/331.8 MB 7.2 MB/s eta 0:00:47
                                              2.5/331.8 MB 7.7 MB/s eta 0:00:43
                                              3.0/331.8 MB 7.5 MB/s eta 0:00:44
                                              3.4/331.8 MB 7.7 MB/s eta 0:00:43
                                              3.7/331.8 MB 7.8 MB/s eta 0:00:43
                                              3


[notice] A new release of pip is available: 23.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
train_df = pd.read_csv("../data/Training_set.csv")
test_df  = pd.read_csv("../data/Testing_set.csv")

Here, we imported the necessary libraries for data manipulation (pandas, numpy), deep learning (tensorflow/keras), and image processing. We then loaded the training and testing CSV files that contained the image filenames and their species labels.

## Image Configuration and Data Preprocessing

In [ ]:
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
train_df = train_df.rename(columns={"Image": "filename", "label": "class"})

train_df_split, val_df_split = train_test_split(
    train_df, test_size=0.2, stratify=train_df['class'], random_state=123
)

We set the image dimensions to 128x128 pixels and batch size to 32 images per training iteration, and renamed the dataframe columns. We split the training data set into 80% training and 20% validation subsets. The stratify parameter makes sure that each butterfly parameter maintains proportional representation in both splits, which is essential for balanced multi-class learning.

## Model Architecture Definition

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)), ...]) 

This chunk constructs a sequential CNN with three convolutional blocks. Each block contains a Conv2D layer (with 32, 64, then 128 filters respectively) followed by MaxPooling2D. After feature extraction, a Flatten layer converts 3D feature maps to 1D, followed by a Dense layer (256 units), Dropout (0.5), and final Dense output layer (75 units with softmax) for class probability prediction. Total: 6,535,307 trainable parameters.

## Model Compilation

In [ ]:
model.compile(
  optimizer="adam",
  loss="categorical_crossentropy",
  metrics=["accuracy"] )

Here, we configure the model for training using the Adam optimizer (adaptive learning rate), categorical cross-entropy loss function (appropriate for multi-class classification with one-hot encoded labels), and accuracy as the evaluation metric.

## Variable Depth CNN Function

In [ ]:
def build_cnn(num_conv_blocks=3,
    base_filters=32, dense_units=256): 
      model = models.Sequential() 
      ...
   return model

This chunk defines a function to construct CNNs with configurable depth. It allows experimentation with different numbers of convolutional blocks while maintaining consistent architecture patterns (i.e. doubling filters each block, same pooling strategy). It also enables systematic comparison of model complexity versus performance.

## Depth Experiment Training Loop

In [ ]:
for d in depths:
    print(f"\nTraining model with {d} conv blocks...\n") model #=
    build_cnn(num_conv_blocks=d) history = model.fit(train_gen,
    validation_data=val_gen, epochs=EPOCHS)

Here, we trained multiple models with 1, 2, and 3 convolutional blocks to analyze the relationship between network depth and classification performance. This chunk stores training history for each configuration, and results show validation accuracy increasing from 31.6% (1 block) to 55.7% (2 blocks), demonstrating the importance of depth for feature abstraction. Training was interrupted before completing the 3-block model.

## Model Training

In [ ]:
EPOCHS = 10 history = model.fit( train_gen,
    validation_data=val_gen, epochs=EPOCHS ) 

This chunk trains the model for 10 epochs using the training generator and validates performance on the validation set after each epoch. Training history records loss and accuracy metrics, showing improvement from 17.8% validation accuracy (epoch 1) to 67.2% (epoch 10).

## Prediction and Export

In [ ]:
pred_probs = model.predict(test_gen)
    pred_indices = np.argmax(pred_probs, axis=1) pred_labels =
    [index_to_class[i] for i in pred_indices] test_df['predicted_label']
    = pred_labels 

Here, we generated predictions on the 2,786 test images by computing class probability distributions, selecting the highest probability class for each image, and mapping numeric indices back to butterfly species names. Predictions are appended to the test dataframe and exported to CSV file maintaining original image order (shuffle=False ensures correct alignment).